<a href="https://colab.research.google.com/github/dimasw09/BelajarKoding/blob/main/Copy_of_Sentiment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install emoji
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.3/356.3 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.5.1-py2.py3-none-any.whl size=351210 sha256=826ff14d3228cee8926732e1681bade347d2414e3efef5f2e0cccce019e200ea
  Stored in directory: /root/.cache/pip/wheels/51/92/44/e2ef13f803aa08711819357e6de0c5fe67b874671141413565
Successfully built emoji
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 14.8 MB/s eta 0:00:00


In [ ]:
!pip install sastrawi
!pip install swifter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 8.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.6/490.6 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 71.4 MB/s eta 0:00:00
  Created wheel for swifter: filename=swifter-1.3.5-py3-none-any.whl size=16518 sha256=e2c5f092fc7c15464293d30a1e344f29b6d098700fdcd7c6dcdedc548e0466e9
  Stored in directory: /root/.cache/pip/wheels/00/7e/8c/438ea309a16ef1a9130849060f962e0d2c4e94b8a4314d5919
Successfully built swifter


In [ ]:
import pandas as pd
import re
import emoji
import contractions
import nltk
import requests
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report

In [ ]:
# Mendapatkan data dari URL
dataUrl = 'https://raw.githubusercontent.com/dimasw09/BelajarKoding/main/DatasetIG.csv'
df = pd.read_csv(dataUrl)

# Membersihkan kolom komentar
def cleaningtext(text):
    text = re.sub('RT\s', '', text) # replace RT tag
    text = re.sub('\B@\w+', '', text) # replace @_username
    text = emoji.demojize(text) # replace emoji with text
    text = re.sub('(http|https):\/\/\S+', '', text) # replace URL
    text = re.sub('#+', '', text) # replace #_something
    text = text.lower() # lower case kalimat
    text = re.sub(r'(.)\1+', r'\1\1', text) # replace word repetition
    text = re.sub(r'[\?\.\!]+(?=[\?.\!])', '', text)
    text = re.sub(r'[0-9]+',' ',text)
    text = re.sub(r"[-()\"#/@;:<>{}'+=~|.!?,_]", " ", text)
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = contractions.fix(text) # replace contractions
    return text

df['Cleaning'] = df['Komentar'].apply(cleaningtext)

# Tokenisasi teks
def tokenizingText(data):
    data = word_tokenize(data)
    return data

df['Tokenizing'] = df['Cleaning'].apply(tokenizingText)

# Mendapatkan kamus slang word dari URL
slangWord_url = 'https://raw.githubusercontent.com/dimasw09/BelajarKoding/main/slang_word.txt'
response = requests.get(slangWord_url)
kamusSlang = eval(response.text)

# Mengubah slang word dalam teks
def convertToSlangword(ulasan):
    pattern = re.compile(r'\b(' + '|'.join(kamusSlang.keys()) + r')\b')
    content = []
    for kata in ulasan:
        filterSlang = pattern.sub(lambda x: kamusSlang[x.group()], kata)
        content.append(filterSlang.lower())
    return content

df['Formalisasi'] = df['Tokenizing'].apply(convertToSlangword)

# Menghapus stop word dalam teks
nltk.download('stopwords')
daftar_stopword = stopwords.words('indonesian')
daftar_stopword = set(daftar_stopword)

def stopwordText(words):
    return [word for word in words if word not in daftar_stopword]

df['Stopword'] = df['Formalisasi'].apply(stopwordText)

# Melakukan stemming pada teks
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemmingText(document):
    return [stemmer.stem(term) for term in document]

df['Stemming'] = df['Stopword'].apply(stemmingText)

# Memisahkan data latih dan data uji
X = df['Stemming']
Y = df['Sentimen']

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Mengubah x_train dan x_test menjadi list
x_train = x_train.apply(lambda x: ' '.join(x)).tolist()
x_test = x_test.apply(lambda x: ' '.join(x)).tolist()

# Membangun model SVM dengan vektorisasi TF-IDF
vectorizer = TfidfVectorizer()
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

svc = SVC()
svc.fit(x_train, y_train)
pred5 = svc.predict(x_test)
print(classification_report(y_test, pred5))

# Melakukan prediksi sentimen pada kalimat baru
kalimat = "Prakerja sangat membantu tapi jelek bangett"
kalimat_cleaned = cleaningtext(kalimat)
kalimat_tokenized = tokenizingText(kalimat_cleaned)
kalimat_formalized = convertToSlangword(kalimat_tokenized)
kalimat_stopword = stopwordText(kalimat_formalized)
kalimat_stemmed = stemmingText(kalimat_stopword)
kalimat_vectorized = vectorizer.transform([' '.join(kalimat_stemmed)])
hasil_sentimen = svc.predict(kalimat_vectorized)[0]

print("Hasil sentimen:", hasil_sentimen)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


              precision    recall  f1-score   support

     Negatif       0.76      0.89      0.82        47
      Netral       0.67      0.50      0.57        20
     Positif       0.95      0.84      0.89        25

    accuracy                           0.79        92
   macro avg       0.79      0.74      0.76        92
weighted avg       0.79      0.79      0.79        92

Hasil sentimen: Negatif
